<a href="https://colab.research.google.com/github/jimwhite/commentator_ai/blob/main/Transcript_to_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q stability-sdk youtube-transcript-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 28.8 MB/s eta 0:00:00


In [2]:
#@title Set up Google Drive for file storage
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    outputs_path = "/content/gdrive/MyDrive/Commentator_AI/Transcript_to_Video"
    !mkdir -p $outputs_path
except:
    outputs_path = "."
print(f"Files will be saved to {outputs_path}")

Mounted at /content/gdrive
Files will be saved to /content/gdrive/MyDrive/Commentator_AI/Transcript_to_Video


In [12]:
#@title Set YouTube URL
import re
def ytIdFromURL(url:str)->str:
   data = re.findall(r"(?:v=|\/)([0-9A-Za-z_-]{11}).*", url)
   if data:
       return data[0]
   return None

YOUTUBE_URL = "https://www.youtube.com/watch?v=vPKp29Luryc" #@param {type:"string"}
video_id = ytIdFromURL(YOUTUBE_URL)
print(f'YouTube video ID = {video_id}')

YouTube video ID = vPKp29Luryc


In [18]:
import json
import os
from youtube_transcript_api import YouTubeTranscriptApi

if not video_id:
  raise ValueError("video_id isn't set")

transcript = None
out_dir = os.path.join(outputs_path, video_id)
transcript_file_path = os.path.join(out_dir, 'transcript.json')
if os.path.exists(transcript_file_path):
  with open(transcript_file_path, 'r') as json_file:
    transcript = json.load(json_file)
    print('Read transcript from file')

if not transcript:
  transcript = YouTubeTranscriptApi.get_transcript(video_id)
  print('Got transcript from YouTube API')
  os.makedirs(out_dir, exist_ok=True)
  with open(transcript_file_path, 'w') as json_file:
    json.dump(transcript, json_file)
    print(f'Saved transcript to file: {transcript_file_path}')

transcript

Read transcript from file


[{'text': 'O say can you see,', 'start': 7.407, 'duration': 4.071},
 {'text': "by the dawn's early light,", 'start': 11.478, 'duration': 3.737},
 {'text': 'What so proudly we hailed', 'start': 15.215, 'duration': 3.837},
 {'text': "at the twilight's last gleaming?",
  'start': 19.052,
  'duration': 3.937},
 {'text': 'Whose broad stripes and bright stars,',
  'start': 22.989,
  'duration': 3.937},
 {'text': 'through the perilous fight,', 'start': 26.926, 'duration': 3.537},
 {'text': "O'er the ramparts we watched,", 'start': 30.463, 'duration': 3.737},
 {'text': 'were so gallantly streaming?', 'start': 34.2, 'duration': 4.071},
 {'text': 'And the rockets red glare,', 'start': 38.271, 'duration': 3.837},
 {'text': 'the bombs bursting in air,', 'start': 42.108, 'duration': 3.837},
 {'text': 'Gave proof through the night', 'start': 45.945, 'duration': 3.737},
 {'text': 'that our flag was still there.',
  'start': 49.682,
  'duration': 3.871},
 {'text': 'O say does that', 'start': 53.553, '

In [ ]:
#@title Connect to the Stability API
import datetime
import getpass
import json
import os
import panel as pn
import param

from base64 import b64encode
from IPython import display
from tqdm import tqdm
from types import SimpleNamespace

from stability_sdk.api import Context
from stability_sdk.animation import AnimationArgs, Animator
from stability_sdk.utils import create_video_from_frames

# @markdown To get your API key visit https://dreamstudio.ai/account
STABILITY_HOST = "grpc.stability.ai:443" #@param {type:"string"}
STABILITY_KEY = getpass.getpass('Enter your API Key')

# Connect to Stability API
context = Context(STABILITY_HOST, STABILITY_KEY)

# Test the connection
context.get_user_info()
print("Connection successful!")